In [2]:
import schedule_Generator as sg
import pandas as pd

Example
- Line 25: 25
- Stop ULB: 5407F
- Direction BOONDAEL GARE
- Date: 08/23

In [3]:
trips = pd.read_csv('../data/gtfs3Sept/trips.csv')
trips_line = trips.loc[trips['route_id']==25,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']=='BOONDAEL GARE',:]
trip_line_head.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
96068,25,236896051,113149920236896051,BOONDAEL GARE,1,8925469,025t0256
96069,25,236896051,113149922236896051,BOONDAEL GARE,1,8925474,025t0256
96072,25,236896051,113149925236896051,BOONDAEL GARE,1,8925469,025t0256
96075,25,236896051,113149929236896051,BOONDAEL GARE,1,8925473,025t0256
96077,25,236896051,113149932236896051,BOONDAEL GARE,1,8925474,025t0256


In [4]:
calendar = pd.read_csv('../data/gtfs3Sept/calendar.csv')
calendar_week = calendar.loc[calendar['start_date']==20210906,:]
calendar_week_day = calendar_week.loc[calendar_week['monday']==1,:]
calendar_week_day.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
15,237645001,1,1,1,1,1,0,0,20210906,20210917
26,237648001,1,1,1,1,1,0,0,20210906,20210917
100,238202001,1,1,1,1,1,0,0,20210906,20210917
203,238119001,1,1,1,1,1,0,0,20210906,20210917
223,238201001,1,1,1,1,1,0,0,20210906,20210917


In [5]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
trip_line_date_head.head()

,route_id,service_id,trip_id
0,25,238202001,113480343238202001
1,25,238202001,113480346238202001
2,25,238202001,113480348238202001
3,25,238202001,113480356238202001
4,25,238202001,113480357238202001


In [8]:
stop_times = pd.read_csv('../data/gtfs3Sept/stop_times.csv')
time_line_date_head = pd.merge(left=trip_line_date_head, right=stop_times, on='trip_id')
time_line_date_head_stop = time_line_date_head.loc[time_line_date_head['stop_id']=='5407F',:]
len(time_line_date_head_stop)

146